In [1]:
def input(path):
    with open(path) as f:
        firstline = f.readline().split()
        N = int(firstline[0])
        D = int(firstline[1])
        alpha = int(firstline[2])
        beta = int(firstline[3])

        person_day_off = []
        for i in range(N):
            line = f.readline()
            person_day_off.append([int(day_off) for day_off in line.split()[:-1]])

        return N, D, alpha, beta, person_day_off

In [2]:
path = 'dataset\data.txt'
N, D, alpha, beta, person_day_off = input(path)

print(N, D, alpha, beta)
print(person_day_off)

9 5 1 3
[[], [4], [], [], [], [], [], [5], []]


# CP

In [86]:
from ortools.sat.python import cp_model
import time

In [87]:
model = cp_model.CpModel()

In [91]:
x = [[model.NewIntVar(0, 4, 'x[' + str(person) + '][' + str(day) + ']') for day in range(D)] for person in range(N)]
y = model.NewIntVar(0, D, 'y')

In [92]:
for day in range(D):
    for shift in range(4):
        model.Add(sum(1 for person in range(N) if x[person][day] == shift) <= beta)
        model.Add(sum(1 for person in range(N) if x[person][day] == shift) >= alpha)
for person in range(N):
    model.Add(sum(1 for day in range(D) if x[person][day] == 3) <= y)

for day in range(D - 1):
    for person in range(N):
        b = model.NewBoolVar('b')
        model.Add(x[person][day] == 3).OnlyEnforceIf(b)
        model.Add(x[person][day] != 3).OnlyEnforceIf(b.Not())
        model.Add(x[person][day + 1] == 4).OnlyEnforceIf(b)

for person, list_day_off in enumerate(person_day_off):
    for day_off in list_day_off:
        model.Add(x[person][day_off - 1] == 4)

In [93]:
model.Minimize(y)
#solver
cp_solver = cp_model.CpSolver()

#solve
start = time.time()
result_status = cp_solver.Solve(model)
end = time.time()
assert result_status == cp_model.OPTIMAL
print('optimal value = ', cp_solver.ObjectiveValue())
print('elapsed time = ', end - start)

for person in range(N):
    for day in range(D):
        print(cp_solver.Value(x[person][day]), end = ' ')
    print()

AssertionError: 